In [60]:
import pandas as pd
import os
from datetime import datetime
from datetime import date


In [61]:

allFiles = os.listdir('scrapedData')
csvFileNames = []
for fileName in allFiles:
    if "csvOut" in fileName:
        csvFileNames.append(fileName)
        

combined_csv = pd.concat( [ pd.read_csv("scrapedData\\" + f) for f in csvFileNames ] )        


In [62]:
pd.options.display.max_columns = None
combined_csv.head()

,regionTrending,trendingRank,timeFetched,videoId,videoTitle,videoCategoryId,videoPublishTime,videoDuration,videoTags,videoViews,videoLikes,videoDislikes,videoCommentCount,videoDescription,vieoLicenced,channelName,channelId,channelDescription,channelPublishedAt,channelViewCount,channelSubsCount,channelVideoCount
0,US,1,2020-07-01 15:09:45.453481,h0U2QUGKbSE,Kanye West – Wash Us In The Blood feat. Travis...,22,2020-06-30T14:00:11Z,PT3M42S,"['kanye', 'kanye west', 'ye', 'yeezus', 'yeezy...",4214043.0,269209.0,13107.0,22665.0,Stream/Download “Wash Us In The Blood” ft. Tra...,False,Kanye West,UCs6eXM7s8Vl5WcECcRHc2qQ,NaN,2006-01-10T22:52:29Z,40986002,6200000,9
1,US,2,2020-07-01 15:09:45.453481,LiB65FQnm6w,Cash vs Flight 1v1 Basketball! Shave Beard or ...,24,2020-06-30T17:53:01Z,PT25M42S,"['cash vs flight 1v1 basketball', 'flight reac...",1497970.0,103096.0,1083.0,10384.0,Get Your Energy Like Me Using GG! \nI receive ...,True,CashNasty,UCvyTdLw8SkVmUcHYXSDEGwA,I make videos. I make you laugh. I be happy.,2013-06-22T01:48:44Z,758385956,3630000,2132
2,US,3,2020-07-01 15:09:45.453481,T8pi91qWnRw,Moneybagg Yo – Said Sum (Official Music Video),10,2020-06-30T16:59:58Z,PT2M59S,NaN,689435.0,51327.0,916.0,1915.0,Moneybagg Yo's new track 'Said Sum' out now: h...,False,MoneyBagg Yo,UCrdPrDuDCbG8xayk5QkRLQA,NaN,2016-10-06T03:39:25Z,344603686,1300000,84
3,US,4,2020-07-01 15:09:45.453481,rAxNSpO78fE,I Surprised Brent Rivera With A Custom iPad Pr...,24,2020-06-30T16:59:43Z,PT13M59S,NaN,2863644.0,212914.0,3235.0,64229.0,"This was Insane, I Can't Believe We Did This F...",True,ZHC,UClQubH2NeMmGLTLgNdLBwXg,Thanks for subscribing! Its my mission to make...,2013-08-07T03:22:54Z,914474888,13700000,272
4,US,5,2020-07-01 15:09:45.453481,mrF-KjnDTxc,"Genoa 1-3 Juventus | Dybala, CR7 & Douglas Cos...",17,2020-06-30T22:30:01Z,PT4M14S,"['Dybala', 'CR7', 'Douglas Costa', 'Genoa', 'D...",4259625.0,106818.0,1644.0,4661.0,Juventus re-open their four point lead at the ...,True,Serie A,UCBJeMCIeLQos7wacox4hmLQ,Welcome to the Official Serie A channel. Over ...,2012-10-30T13:54:30Z,1406914395,4660000,18934


In [63]:
pd.options.display.max_columns = None
#combined_csv = pd.read_csv("scrapedData\\" + csvFileNames[0])
numRows = len(combined_csv.index)
print(numRows)
columnCountCurrent = (len(combined_csv.index))
notAvailableColumn = ["notAvailable"] * numRows
newStringColumn = ["new"] * numRows
combined_csv = combined_csv.rename({'vieoLicenced': 'videoLicensed'}, axis=1)

combined_csv['thumbnail_link'] = notAvailableColumn
combined_csv['comments_disabled'] = notAvailableColumn
combined_csv['ratings_disabled'] = notAvailableColumn
combined_csv['video_error_or_removed'] = notAvailableColumn

# Creating publishedDateCorrectFormat column
videoPublishTimeColumn = (combined_csv['videoPublishTime'])
videoPublishTimeColumnSplit1 = [i.split('T', 1)[0] for i in videoPublishTimeColumn]
videoPublishTimeZ = [i.split('T', 1)[1] for i in videoPublishTimeColumn]
videoPublishTimeZFinal = [s.replace('Z', '') for s in videoPublishTimeZ]
#Converting the Z time into float
videoPublishTimeZFloat = [float(i.split(':', 2)[0]) + float(i.split(':', 2)[1])/60 + float(i.split(':', 2)[2])/3600 for i in videoPublishTimeZFinal]

videoPublishTimeStandard = [datetime.strptime(i, '%Y-%m-%d') for i in videoPublishTimeColumnSplit1]    # THis is in standard datetime object format
publishedDayOfWeek = [((i.weekday()+1)%7)+1 for i in videoPublishTimeStandard]

publishedDateCorrectFormat = [x.strftime("%d-%m-%y") for x in videoPublishTimeStandard]
combined_csv['publishedDateCorrectFormat'] = publishedDateCorrectFormat




# Creating trendingDateCorrectFormat
timeFetchedColumn = combined_csv['timeFetched']
timeFetchedColumnSplit1 = [i.split(' ', 1)[0] for i in timeFetchedColumn]
timeFetchedStandard = [datetime.strptime(i, '%Y-%m-%d') for i in timeFetchedColumnSplit1]
trendingDateCorrectFormat = [x.strftime("%d-%m-%y") for x in timeFetchedStandard]
combined_csv['trendingDateCorrectFormat'] = trendingDateCorrectFormat


# Calculating day difference
dayDifference = [(timeFetchedStandard[i] - videoPublishTimeStandard[i]).days for i in range(numRows)]
combined_csv['dayDifference'] = dayDifference

# publishedZTime column - format hh:mm:ss
combined_csv['publishedZTime'] = videoPublishTimeZFinal

# publishedZTimeFloat column - converted to float
combined_csv['publishedZTimeFloat'] = videoPublishTimeZFloat

# publishedDayOfWeek column - 1 - sunday , 7 - saturday
combined_csv['publishedDayOfWeek'] = publishedDayOfWeek

# newOrOldData column
combined_csv['newOrOldData'] = newStringColumn


combined_csv.head()

11400


,regionTrending,trendingRank,timeFetched,videoId,videoTitle,videoCategoryId,videoPublishTime,videoDuration,videoTags,videoViews,videoLikes,videoDislikes,videoCommentCount,videoDescription,videoLicensed,channelName,channelId,channelDescription,channelPublishedAt,channelViewCount,channelSubsCount,channelVideoCount,thumbnail_link,comments_disabled,ratings_disabled,video_error_or_removed,publishedDateCorrectFormat,trendingDateCorrectFormat,dayDifference,publishedZTime,publishedZTimeFloat,publishedDayOfWeek,newOrOldData
0,US,1,2020-07-01 15:09:45.453481,h0U2QUGKbSE,Kanye West – Wash Us In The Blood feat. Travis...,22,2020-06-30T14:00:11Z,PT3M42S,"['kanye', 'kanye west', 'ye', 'yeezus', 'yeezy...",4214043.0,269209.0,13107.0,22665.0,Stream/Download “Wash Us In The Blood” ft. Tra...,False,Kanye West,UCs6eXM7s8Vl5WcECcRHc2qQ,NaN,2006-01-10T22:52:29Z,40986002,6200000,9,notAvailable,notAvailable,notAvailable,notAvailable,30-06-20,01-07-20,1,14:00:11,14.003056,3,new
1,US,2,2020-07-01 15:09:45.453481,LiB65FQnm6w,Cash vs Flight 1v1 Basketball! Shave Beard or ...,24,2020-06-30T17:53:01Z,PT25M42S,"['cash vs flight 1v1 basketball', 'flight reac...",1497970.0,103096.0,1083.0,10384.0,Get Your Energy Like Me Using GG! \nI receive ...,True,CashNasty,UCvyTdLw8SkVmUcHYXSDEGwA,I make videos. I make you laugh. I be happy.,2013-06-22T01:48:44Z,758385956,3630000,2132,notAvailable,notAvailable,notAvailable,notAvailable,30-06-20,01-07-20,1,17:53:01,17.883611,3,new
2,US,3,2020-07-01 15:09:45.453481,T8pi91qWnRw,Moneybagg Yo – Said Sum (Official Music Video),10,2020-06-30T16:59:58Z,PT2M59S,NaN,689435.0,51327.0,916.0,1915.0,Moneybagg Yo's new track 'Said Sum' out now: h...,False,MoneyBagg Yo,UCrdPrDuDCbG8xayk5QkRLQA,NaN,2016-10-06T03:39:25Z,344603686,1300000,84,notAvailable,notAvailable,notAvailable,notAvailable,30-06-20,01-07-20,1,16:59:58,16.999444,3,new
3,US,4,2020-07-01 15:09:45.453481,rAxNSpO78fE,I Surprised Brent Rivera With A Custom iPad Pr...,24,2020-06-30T16:59:43Z,PT13M59S,NaN,2863644.0,212914.0,3235.0,64229.0,"This was Insane, I Can't Believe We Did This F...",True,ZHC,UClQubH2NeMmGLTLgNdLBwXg,Thanks for subscribing! Its my mission to make...,2013-08-07T03:22:54Z,914474888,13700000,272,notAvailable,notAvailable,notAvailable,notAvailable,30-06-20,01-07-20,1,16:59:43,16.995278,3,new
4,US,5,2020-07-01 15:09:45.453481,mrF-KjnDTxc,"Genoa 1-3 Juventus | Dybala, CR7 & Douglas Cos...",17,2020-06-30T22:30:01Z,PT4M14S,"['Dybala', 'CR7', 'Douglas Costa', 'Genoa', 'D...",4259625.0,106818.0,1644.0,4661.0,Juventus re-open their four point lead at the ...,True,Serie A,UCBJeMCIeLQos7wacox4hmLQ,Welcome to the Official Serie A channel. Over ...,2012-10-30T13:54:30Z,1406914395,4660000,18934,notAvailable,notAvailable,notAvailable,notAvailable,30-06-20,01-07-20,1,22:30:01,22.500278,3,new


In [64]:
combined_csv.to_csv( "newDataOnly_csv_newFormat.csv", index=False )

In [65]:
oldData = pd.read_excel (r'finalOldData.xlsx')

In [66]:
combinedOldNew_csv = pd.concat( [oldData, combined_csv] )        
combinedOldNew_csv.to_csv( "oldAndNewData.csv", index=False )

In [67]:
combinedOldNew_csv.head()

,regionTrending,trendingRank,timeFetched,videoId,videoTitle,videoCategoryId,videoPublishTime,videoDuration,videoTags,videoViews,videoLikes,videoDislikes,videoCommentCount,videoDescription,videoLicensed,channelName,channelId,channelDescription,channelPublishedAt,channelViewCount,channelSubsCount,channelVideoCount,thumbnail_link,comments_disabled,ratings_disabled,video_error_or_removed,publishedDateCorrectFormat,trendingDateCorrectFormat,dayDifference,publishedZTime,publishedZTimeFloat,publishedDayOfWeek,newOrOldData
0,US,0,2017-11-14 00:00:00,2kyS6SvSYSE,WE WANT TO TALK ABOUT OUR MARRIAGE,22,2017-11-13T17:13:01.000Z,notAvailable,SHANtell martin,748374.0,57527.0,2966.0,15954.0,SHANTELL'S CHANNEL - https://www.youtube.com/s...,notAvailable,CaseyNeistat,0,notAvailable,notAvailable,0,0,0,https://i.ytimg.com/vi/2kyS6SvSYSE/default.jpg,False,False,False,2017-11-13 00:00:00,2017-11-14 00:00:00,1,17:13:01,17.216944,2,old
1,US,0,2017-11-14 00:00:00,1ZAPwfrtAFY,The Trump Presidency: Last Week Tonight with J...,24,2017-11-13T07:30:00.000Z,notAvailable,"last week tonight trump presidency|""last week ...",2418783.0,97185.0,6146.0,12703.0,"One year after the presidential election, John...",notAvailable,LastWeekTonight,0,notAvailable,notAvailable,0,0,0,https://i.ytimg.com/vi/1ZAPwfrtAFY/default.jpg,False,False,False,2017-11-13 00:00:00,2017-11-14 00:00:00,1,07:30:00,7.500000,2,old
2,US,0,2017-11-14 00:00:00,5qpjK5DgCt4,"Racist Superman | Rudy Mancuso, King Bach & Le...",23,2017-11-12T19:05:24.000Z,notAvailable,"racist superman|""rudy""|""mancuso""|""king""|""bach""...",3191434.0,146033.0,5339.0,8181.0,WATCH MY PREVIOUS VIDEO â–¶ \n\nSUBSCRIBE â–º ...,notAvailable,Rudy Mancuso,0,notAvailable,notAvailable,0,0,0,https://i.ytimg.com/vi/5qpjK5DgCt4/default.jpg,False,False,False,2017-11-12 00:00:00,2017-11-14 00:00:00,2,19:05:24,19.090000,1,old
3,US,0,2017-11-14 00:00:00,puqaWrEC7tY,Nickelback Lyrics: Real or Fake?,24,2017-11-13T11:00:04.000Z,notAvailable,"rhett and link|""gmm""|""good mythical morning""|""...",343168.0,10172.0,666.0,2146.0,Today we find out if Link is a Nickelback amat...,notAvailable,Good Mythical Morning,0,notAvailable,notAvailable,0,0,0,https://i.ytimg.com/vi/puqaWrEC7tY/default.jpg,False,False,False,2017-11-13 00:00:00,2017-11-14 00:00:00,1,11:00:04,11.001111,2,old
4,US,0,2017-11-14 00:00:00,d380meD0W0M,I Dare You: GOING BALD!?,24,2017-11-12T18:01:41.000Z,notAvailable,"ryan|""higa""|""higatv""|""nigahiga""|""i dare you""|""...",2095731.0,132235.0,1989.0,17518.0,I know it's been a while since we did this sho...,notAvailable,nigahiga,0,notAvailable,notAvailable,0,0,0,https://i.ytimg.com/vi/d380meD0W0M/default.jpg,False,False,False,2017-11-12 00:00:00,2017-11-14 00:00:00,2,18:01:41,18.028056,1,old
